In [29]:
import numpy as np
import awswrangler as wr

In [30]:
import pandas as pd, requests, boto3, pyarrow as pa, pyarrow.parquet as pq

In [31]:
s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

aaa-iceberg-validator
aaaa-iceberg
anythingwild
aws-athena-query-results-340752803932-eu-central-1
ayodeji-data-ingestion-bucket
ayodeji-state-bucket-data-ingestion
boto-awswrangler-demo
cde-athena-query-result
cde-aws-wrangler-demo
cde-blessed-madukoma-s3-bucket
cde-bukky-bucket
cde-capstone
cde-capstone-bucket-sam
cde-ebun-bucket
cde-hakeem-backend-bucket
cde-hakeem-bucket
cde-hakeem-datalake-bucket
cde-hammed-adegunju
cde-leo-bucket
cde-pasky-bucket
cde-peace-bucket
cde-practice-travel-agency-bucket
cde-raw-data
cde-terraform-state
cde-test-tinubu
cde-your-s3-bucket-name
cdechiomabucket
chisomnwa-bucket
chisomnwa-travel-agency-backend-bucket
dami-cde-bucket
datawarehouse4me
dev-cde-travel-agency
dev-male-female-ingestion
dpe-custom-bucket
dpe-terraform-statefile
edsbucket7
femmyte-cde-bucket
greybuck1
iceberg-validator
leo-cde-project
leo-cde-project-backend
leo-dami
mayo-iceberg-demo
my-tf-female-bucket
olubusayo-bkt
olubusayo-blessing-friend
random-profile-terraform-state-bucket
r

In [32]:
url = 'https://restcountries.com/v3.1/all?fields=name,independent,unMember,startOfWeek,currencies,idd,country_code_root,languages,capital,continents,country_code_suffixes,region,subregion'
r = requests.get(url)

In [33]:
r.json()

[{'startOfWeek': 'monday',
  'name': {'common': 'South Georgia',
   'official': 'South Georgia and the South Sandwich Islands',
   'nativeName': {'eng': {'official': 'South Georgia and the South Sandwich Islands',
     'common': 'South Georgia'}}},
  'independent': False,
  'unMember': False,
  'currencies': {'SHP': {'name': 'Saint Helena pound', 'symbol': '£'}},
  'idd': {'root': '+5', 'suffixes': ['00']},
  'capital': ['King Edward Point'],
  'region': 'Antarctic',
  'subregion': '',
  'languages': {'eng': 'English'},
  'continents': ['Antarctica']},
 {'startOfWeek': 'monday',
  'name': {'common': 'Grenada',
   'official': 'Grenada',
   'nativeName': {'eng': {'official': 'Grenada', 'common': 'Grenada'}}},
  'independent': True,
  'unMember': True,
  'currencies': {'XCD': {'name': 'Eastern Caribbean dollar', 'symbol': '$'}},
  'idd': {'root': '+1', 'suffixes': ['473']},
  'capital': ["St. George's"],
  'region': 'Americas',
  'subregion': 'Caribbean',
  'languages': {'eng': 'English'}

In [34]:
df = pd.DataFrame(r.json())

In [35]:
df.columns

Index(['startOfWeek', 'name', 'independent', 'unMember', 'currencies', 'idd',
       'capital', 'region', 'subregion', 'languages', 'continents'],
      dtype='object')

In [37]:
wr.s3.to_parquet(
    df=df.copy(),
    path="s3://dami-cde-bucket/raw/raw_data.parquet"
)


{'paths': ['s3://dami-cde-bucket/raw/raw_data.parquet'],
 'partitions_values': {}}

In [39]:
wr.s3.to_parquet(
    df=trs_df,
    path="s3://dami-cde-bucket/staged/stage.parquet"
)


ArrowTypeError: Expected bytes, got a 'numpy.ndarray' object

In [18]:
def extract_value(d, key,islist=False):
    try:
        value = d.get(key)
        if islist:
            return value[0]
        else:
            return value
    except:
        return None

In [19]:
df = wr.s3.read_parquet(
    path='s3://dami-cde-bucket/raw/raw_data.parquet'
)

In [20]:
df

,startOfWeek,name,independent,unMember,currencies,idd,capital,region,subregion,languages,continents
0,monday,"{'common': 'South Georgia', 'nativeName': {'af...",False,False,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+5', 'suffixes': ['00']}",[King Edward Point],Antarctic,,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Antarctica]
1,monday,"{'common': 'Grenada', 'nativeName': {'afr': No...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+1', 'suffixes': ['473']}",[St. George's],Americas,Caribbean,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[North America]
2,monday,"{'common': 'Switzerland', 'nativeName': {'afr'...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+4', 'suffixes': ['1']}",[Bern],Europe,Western Europe,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Europe]
3,monday,"{'common': 'Sierra Leone', 'nativeName': {'afr...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+2', 'suffixes': ['32']}",[Freetown],Africa,Western Africa,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Africa]
4,monday,"{'common': 'Hungary', 'nativeName': {'afr': No...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+3', 'suffixes': ['6']}",[Budapest],Europe,Central Europe,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Europe]
...,...,...,...,...,...,...,...,...,...,...,...
245,monday,"{'common': 'Belgium', 'nativeName': {'afr': No...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+3', 'suffixes': ['2']}",[Brussels],Europe,Western Europe,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Europe]
246,sunday,"{'common': 'Israel', 'nativeName': {'afr': Non...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+9', 'suffixes': ['72']}",[Jerusalem],Asia,Western Asia,"{'afr': None, 'amh': None, 'ara': 'Arabic', 'a...",[Asia]
247,monday,"{'common': 'New Zealand', 'nativeName': {'afr'...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+6', 'suffixes': ['4']}",[Wellington],Oceania,Australia and New Zealand,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[Oceania]
248,monday,"{'common': 'Nicaragua', 'nativeName': {'afr': ...",True,True,"{'AED': None, 'AFN': None, 'ALL': None, 'AMD':...","{'root': '+5', 'suffixes': ['05']}",[Managua],Americas,Central America,"{'afr': None, 'amh': None, 'ara': None, 'arc':...",[North America]


In [21]:
def get_nested_values(d,return_type='val'):
    try:
        for key, value in d.items():
            if value is not None:
                if return_type == 'code':
                    return key
                elif return_type == 'name':
                    return value['name']
                elif return_type == 'symbol':
                    return value['symbol']
                else:
                    return value
        return None
    except:
        return None

In [12]:
x = ['a','b']

In [13]:
str(x)

"['a', 'b']"

In [22]:
#df['common_name'] = df['name'].apply(value_extract('common',col=df['name'])) 

# Key to extract

# Create a new column with extracted values
df['country_name'] = df['name'].apply(lambda d: extract_value(d, 'common'))
df['independence'] = df['independent']

df['united_nation_members'] = df['unMember']

df['startofweek'] = df['startOfWeek']

df['official_country_name'] = df['name'].apply(lambda d: extract_value(d, 'official'))

df['common_native_name'] = df['name'].apply(lambda d: extract_value(d, 'official'))

df['currency_code'] = df['currencies'].apply(lambda d: get_nested_values(d, 'code'))

df['currency_name'] = df['currencies'].apply(lambda d: get_nested_values(d, 'name'))

df['currency_symbol'] = df['currencies'].apply(lambda d: get_nested_values(d, 'symbol'))

df['country_code_root'] = df['idd'].apply(lambda d: extract_value(d, 'root'))

df['country_code_suffixes'] = df['idd'].apply(lambda d: extract_value(d, 'suffixes'))

df['len_code'] = df.country_code_suffixes.apply(lambda x : len(x))

df['country_code_suffixes'] = df.apply(
    lambda row: row['country_code_suffixes'][0] if row['len_code'] >= 1 else row['country_code_suffixes'], 
    axis=1
)

df['country_code_root'] = df['country_code_root'].apply(lambda x: str(x))


df['country_code_idd'] = df['country_code_root'] + df.country_code_suffixes 

df['len_code'] = df.capital.apply(lambda x : len(x))

df['capital'] = df.apply(
    lambda row: row['capital'][0] if row['len_code'] >= 1 else row['capital'], 
    axis=1
)

df['languages'] = df['languages'].apply(lambda d: get_nested_values(d))

df['len_code'] = df.continents.apply(lambda x : len(x))

df['continents'] = df.apply(
    lambda row: row['continents'][0] if row['len_code'] >= 1 else row['continents'], 
    axis=1
)


In [23]:
new_cols = [
    'country_name','independence','united_nation_members','startofweek','official_country_name','common_native_name','currency_code',
    'currency_name','currency_symbol','country_code_idd','capital','region','subregion','languages','continents'
    ]

df = df[new_cols]

In [24]:
df

,country_name,independence,united_nation_members,startofweek,official_country_name,common_native_name,currency_code,currency_name,currency_symbol,country_code_idd,capital,region,subregion,languages,continents
0,South Georgia,False,False,monday,South Georgia and the South Sandwich Islands,South Georgia and the South Sandwich Islands,SHP,Saint Helena pound,£,+500,King Edward Point,Antarctic,,English,Antarctica
1,Grenada,True,True,monday,Grenada,Grenada,XCD,Eastern Caribbean dollar,$,+1473,St. George's,Americas,Caribbean,English,North America
2,Switzerland,True,True,monday,Swiss Confederation,Swiss Confederation,CHF,Swiss franc,Fr.,+41,Bern,Europe,Western Europe,French,Europe
3,Sierra Leone,True,True,monday,Republic of Sierra Leone,Republic of Sierra Leone,SLL,Sierra Leonean leone,Le,+232,Freetown,Africa,Western Africa,English,Africa
4,Hungary,True,True,monday,Hungary,Hungary,HUF,Hungarian forint,Ft,+36,Budapest,Europe,Central Europe,Hungarian,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Belgium,True,True,monday,Kingdom of Belgium,Kingdom of Belgium,EUR,Euro,€,+32,Brussels,Europe,Western Europe,German,Europe
246,Israel,True,True,sunday,State of Israel,State of Israel,ILS,Israeli new shekel,₪,+972,Jerusalem,Asia,Western Asia,Arabic,Asia
247,New Zealand,True,True,monday,New Zealand,New Zealand,NZD,New Zealand dollar,$,+64,Wellington,Oceania,Australia and New Zealand,English,Oceania
248,Nicaragua,True,True,monday,Republic of Nicaragua,Republic of Nicaragua,NIO,Nicaraguan córdoba,C$,+505,Managua,Americas,Central America,Spanish,North America


In [109]:
df.columns

Index(['country_name', 'independence', 'united_nation_members', 'startofweek',
       'official_country_name', 'common_native_name', 'currency_code',
       'currency_name', 'currency_symbol', 'country_code_idd', 'capital',
       'region', 'subregion', 'languages', 'continents'],
      dtype='object')

In [27]:
trs_df = df.copy()

In [45]:
trs_df = trs_df.astype(str)

In [46]:
wr.s3.to_parquet(
    df=trs_df,
    path="s3://dami-cde-bucket/stage/staged_data.parquet"
)


{'paths': ['s3://dami-cde-bucket/stage/staged_data.parquet'],
 'partitions_values': {}}

In [43]:
trs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country_name           250 non-null    object 
 1   independence           249 non-null    boolean
 2   united_nation_members  250 non-null    boolean
 3   startofweek            250 non-null    string 
 4   official_country_name  250 non-null    object 
 5   common_native_name     250 non-null    object 
 6   currency_code          247 non-null    object 
 7   currency_name          247 non-null    object 
 8   currency_symbol        247 non-null    object 
 9   country_code_idd       250 non-null    object 
 10  capital                250 non-null    object 
 11  region                 250 non-null    string 
 12  subregion              250 non-null    string 
 13  languages              249 non-null    object 
 14  continents             250 non-null    object 
dtypes: boo

In [41]:
for col in trs_df.columns:
    if isinstance(trs_df[col].iloc[0], np.ndarray):
        print(f"Column '{col}' contains numpy.ndarray")

In [47]:
wr.s3.read_parquet(
    path='s3://dami-cde-bucket/stage/staged_data.parquet'
)

,country_name,independence,united_nation_members,startofweek,official_country_name,common_native_name,currency_code,currency_name,currency_symbol,country_code_idd,capital,region,subregion,languages,continents
0,South Georgia,False,False,monday,South Georgia and the South Sandwich Islands,South Georgia and the South Sandwich Islands,SHP,Saint Helena pound,£,+500,King Edward Point,Antarctic,,English,Antarctica
1,Grenada,True,True,monday,Grenada,Grenada,XCD,Eastern Caribbean dollar,$,+1473,St. George's,Americas,Caribbean,English,North America
2,Switzerland,True,True,monday,Swiss Confederation,Swiss Confederation,CHF,Swiss franc,Fr.,+41,Bern,Europe,Western Europe,French,Europe
3,Sierra Leone,True,True,monday,Republic of Sierra Leone,Republic of Sierra Leone,SLL,Sierra Leonean leone,Le,+232,Freetown,Africa,Western Africa,English,Africa
4,Hungary,True,True,monday,Hungary,Hungary,HUF,Hungarian forint,Ft,+36,Budapest,Europe,Central Europe,Hungarian,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Belgium,True,True,monday,Kingdom of Belgium,Kingdom of Belgium,EUR,Euro,€,+32,Brussels,Europe,Western Europe,German,Europe
246,Israel,True,True,sunday,State of Israel,State of Israel,ILS,Israeli new shekel,₪,+972,Jerusalem,Asia,Western Asia,Arabic,Asia
247,New Zealand,True,True,monday,New Zealand,New Zealand,NZD,New Zealand dollar,$,+64,Wellington,Oceania,Australia and New Zealand,English,Oceania
248,Nicaragua,True,True,monday,Republic of Nicaragua,Republic of Nicaragua,NIO,Nicaraguan córdoba,C$,+505,Managua,Americas,Central America,Spanish,North America
